# Script for a period

In [162]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [163]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [164]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [165]:
data_giper

{'adverts': [{'type': 8,
   'status': 11,
   'count': 4,
   'advert_list': [{'advertId': 21526558,
     'changeTime': '2025-02-07T16:43:04.178341+03:00'},
    {'advertId': 22844037, 'changeTime': '2025-01-31T13:40:18.926964+03:00'},
    {'advertId': 22844946, 'changeTime': '2025-03-04T12:36:41.470052+03:00'},
    {'advertId': 23402220, 'changeTime': '2025-03-04T13:29:43.833312+03:00'}]},
  {'type': 9,
   'status': 9,
   'count': 5,
   'advert_list': [{'advertId': 22950405,
     'changeTime': '2025-03-03T12:03:56.844328+03:00'},
    {'advertId': 22950831, 'changeTime': '2025-03-03T12:28:57.920554+03:00'},
    {'advertId': 22951156, 'changeTime': '2025-03-03T15:35:29.753881+03:00'},
    {'advertId': 22951268, 'changeTime': '2025-02-07T16:45:41.628974+03:00'},
    {'advertId': 22954069, 'changeTime': '2025-02-07T16:05:53.370485+03:00'}]},
  {'type': 8,
   'status': 7,
   'count': 7,
   'advert_list': [{'advertId': 17741906,
     'changeTime': '2024-06-08T10:38:13.264875+03:00'},
    {'adv

In [166]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [167]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [168]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[213920, 213946, 213954, 225156, 232543, 232547, 238682, 244654, 244667, 354788, 354832, 354853, 535702, 535721, 535733, 535812, 535864, 535918, 535940, 535963, 535994, 536002, 536018, 536060, 536362, 536369, 536385, 536415, 536427, 536485, 536514, 536537, 536549, 536568, 536610, 536617, 536631, 536646, 536659, 788466, 788472, 788580, 788586, 788636, 788641, 789178, 789191, 953420, 953764, 953774], [958835, 960520, 960525, 960529, 960544, 960551, 960611, 960618, 960621, 960624, 960626, 960675, 960682, 960689, 960693, 960701, 1005700, 1320657, 1320668, 1320684, 2594125, 2594226, 2594682, 2594752, 2595105, 2595113, 2851221, 2851300, 2851888, 2852058, 3467203, 3474308, 3474441, 3476519, 3476542, 3476739, 3476757, 3476880, 3476891, 3477028, 4558192, 4981861, 5001772, 5146248, 5146392, 5149382, 6626942, 6923910, 6923977, 7202606], [7202975, 7203114, 7339607, 7339693, 7339737, 7339839, 7339928, 7339962, 7340006, 7340088, 7414147, 8779205, 8779248, 8779408, 

In [169]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-14,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [241]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2024-11-01",  # Replace with your start date
    "end": "2024-11-30"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [242]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
#with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    #json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [243]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [244]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [245]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [246]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,4667,113,2.42,9.57,1081.36,12,3,2.65,3,7936,"[{'date': '2024-11-26T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 171562...",21624823,2024-11-25,2024-11-30
1,111762,1949,1.74,4.78,9320.92,152,31,1.59,31,56828,"[{'date': '2024-11-19T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 137609...",21498159,2024-11-19,2024-11-30
2,7071,141,1.99,12.27,1729.68,4,0,0.00,0,0,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 257314...",19994021,2024-11-01,2024-11-30
3,93284,3261,3.50,2.57,8369.55,221,40,1.23,41,71715,"[{'date': '2024-11-19T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 137609...",21497844,2024-11-19,2024-11-30
4,19759,1186,6.00,3.59,4262.65,80,10,0.84,10,125796,"[{'date': '2024-11-07T03:00:00+03:00', 'views'...",NaN,21204285,2024-11-07,2024-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,11940,700,5.86,1.96,1371.03,33,4,0.57,4,28784,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 260854...",18870595,2024-11-01,2024-11-30
71,53421,2157,4.04,2.56,5522.30,143,36,1.67,38,164049,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 209076...",18870619,2024-11-01,2024-11-30
72,88257,6037,6.84,1.37,8242.88,601,206,3.41,268,149570,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 224944...",16530753,2024-11-01,2024-11-30
73,25895,1104,4.26,2.06,2276.34,29,4,0.36,4,23909,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 203510...",18447862,2024-11-01,2024-11-30


In [233]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

71


In [247]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2024-11-26 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,16,0,0.00,0.00,4.31,0,0,0.0,0,0,21624823
1,2024-11-26 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,9,0,0.00,0.00,2.21,0,0,0.0,0,0,21624823
2,2024-11-26 03:00:00,171562375,Ингалятор небулайзер компрессорный детский CN-...,92,0,0.00,0.00,25.27,0,0,0.0,0,0,21624823
3,2024-11-26 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,188,3,1.60,17.70,53.11,1,0,0.0,0,0,21624823
4,2024-11-26 03:00:00,171562370,Ингалятор небулайзер компрессорный детский CN-...,76,2,2.63,10.67,21.33,0,0,0.0,0,0,21624823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15376,2024-11-29 03:00:00,25963255,Кухонные весы L 15,4,0,0.00,0.00,0.48,0,0,0.0,0,0,18870701
15377,2024-11-29 03:00:00,25963255,Кухонные весы L 15,11,1,9.09,1.36,1.36,0,0,0.0,0,0,18870701
15378,2024-11-29 03:00:00,25963255,Кухонные весы L 15,4,0,0.00,0.00,0.64,0,0,0.0,0,0,18870701
15379,2024-11-30 03:00:00,25963255,Кухонные весы L 15,13,0,0.00,0.00,0.86,0,0,0.0,0,0,18870701


In [248]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25914858,2024-11-01 03:00:00,Яйцеварка электрическая E 7,323,6,1.216667,4.926667,38.59,0,0,0.000000,0,0,18870619,WB-GutenTech,Wildberries
1,25963255,2024-11-01 03:00:00,Кухонные весы L 15,166,9,3.500000,1.916667,19.04,0,2,37.500000,2,7054,18870701,WB-GutenTech,Wildberries
2,26085467,2024-11-01 03:00:00,Вакууматор для продуктов с запайкой VC 10,291,15,5.700000,1.170000,29.07,0,0,0.000000,0,0,18870595,WB-GutenTech,Wildberries
3,46292575,2024-11-01 03:00:00,Яйцеварка электрическая автоматическая E 9,602,25,4.666667,2.556667,65.66,0,1,11.110000,1,4957,18870619,WB-GutenTech,Wildberries
4,64331255,2024-11-01 03:00:00,Электрическая зубная щетка взрослая Pro 3 3500...,667,33,4.246667,1.260000,30.21,0,2,4.316667,2,7826,20139198,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,288465478,2024-11-30 03:00:00,Набор Breeze бритва с кассетой и шампунь 250мл,204,3,1.070000,11.620000,50.51,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
7285,288465480,2024-11-30 03:00:00,Набор Gillette Venus с гелем для бритья,340,10,3.246667,7.906667,76.96,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
7286,288465481,2024-11-30 03:00:00,Набор Gillette Venus с чехлом для бритвы,351,15,3.223333,3.100000,77.30,0,0,0.000000,0,0,21693448,WB-Smart-Market,Wildberries
7287,288465484,2024-11-30 03:00:00,Набор Mach3 с лосьоном после бритья,613,18,2.380000,11.806667,146.66,0,1,2.380000,1,911,21693077,WB-Smart-Market,Wildberries


In [236]:
""" # Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_grouped_combined_campaigns.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.") """

DataFrame exported to df_guten_data.xlsx successfully.


In [249]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [250]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25914858,2024-11-01 03:00:00,Яйцеварка электрическая E 7,323,6,38.59,0,0,0,0,18870619,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Яйцеварки_3 SKU_25.07.24,Идут показы,Автоматическая кампания
1,25963255,2024-11-01 03:00:00,Кухонные весы L 15,166,9,19.04,0,2,2,7054,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
2,26085467,2024-11-01 03:00:00,Вакууматор для продуктов с запайкой VC 10,291,15,29.07,0,0,0,0,18870595,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания
3,46292575,2024-11-01 03:00:00,Яйцеварка электрическая автоматическая E 9,602,25,65.66,0,1,1,4957,18870619,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Яйцеварки_3 SKU_25.07.24,Идут показы,Автоматическая кампания
4,64331255,2024-11-01 03:00:00,Электрическая зубная щетка взрослая Pro 3 3500...,667,33,30.21,0,2,2,7826,20139198,WB-GutenTech,Wildberries,2100-01-01,2024-09-24,2024-09-24,ORAL-B_Щетки 3 SKU_24.09.2024,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284,288465478,2024-11-30 03:00:00,Набор Breeze бритва с кассетой и шампунь 250мл,204,3,50.51,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
7285,288465480,2024-11-30 03:00:00,Набор Gillette Venus с гелем для бритья,340,10,76.96,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
7286,288465481,2024-11-30 03:00:00,Набор Gillette Venus с чехлом для бритвы,351,15,77.30,0,0,0,0,21693448,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2024-11-28,VENUS_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания
7287,288465484,2024-11-30 03:00:00,Набор Mach3 с лосьоном после бритья,613,18,146.66,0,1,1,911,21693077,WB-Smart-Market,Wildberries,2100-01-01,2024-11-28,2025-02-17,Gillette_Наборы_28.11.2024,Кампания на паузе,Автоматическая кампания


In [224]:
""" # Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_final.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.") """

DataFrame exported to df_guten_data.xlsx successfully.


## Insert the data

In [ ]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [252]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
